In [56]:
import pylab as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
import scipy
from scipy import interpolate
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.interpolate import interp2d
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import re
import sys
import glob
import random
try:
    import cPickle as pickle
except ImportError:
    import pickle

In [57]:
r=0

# Pull in all of the MCeQoutputs to put into one dataframe

# infilepath = "/Users/mistryk2/Packages/MCEq/workdir/MCeQouput*.h5"
infilepath = "/Users/mistryk2/Packages/MCEq/outputfiles/*s.h5"

for files in glob.glob(infilepath):

    if r==0:
        mussimData=pd.read_hdf(files)
        r+=1
    else:
        df=pd.read_hdf(files)
        mussimData=mussimData.append(df, ignore_index=True)

# Rename the columns
mussimData=mussimData.rename(columns={"theta": "alpha", "phis":  "beta"})

display(mussimData)

# Convert the columns from deg to rad
mussimData.beta=np.deg2rad(mussimData.beta)
mussimData.alpha=np.deg2rad(mussimData.alpha)

,MuFLux,Energies,alpha,columndepth,altitude,beta
0,3.689041e-08,8.912509e-02,64.171273,2328.404852,28954.196303,-59.014653
1,8.296956e-08,1.122018e-01,64.171273,2328.404852,28954.196303,-59.014653
2,1.859312e-07,1.412538e-01,64.171273,2328.404852,28954.196303,-59.014653
3,4.134257e-07,1.778279e-01,64.171273,2328.404852,28954.196303,-59.014653
4,9.132302e-07,2.238721e-01,64.171273,2328.404852,28954.196303,-59.014653
...,...,...,...,...,...,...
40530,5.728973e-14,3.548134e+10,44.690708,1385.893332,56072.589419,-123.758884
40531,2.379178e-15,4.466836e+10,44.690708,1385.893332,56072.589419,-123.758884
40532,-2.452037e-15,5.623413e+10,44.690708,1385.893332,56072.589419,-123.758884
40533,-1.895575e-16,7.079458e+10,44.690708,1385.893332,56072.589419,-123.758884


In [58]:
# Need a list of alphas and betas to interpolate between
listofpoints_alpha = []
listofpoints_beta = []
tmpdict = {}
alphas  = mussimData.alpha
betas   = mussimData.beta

for i in range(0,len(alphas)):
     tmpdict[alphas[i],betas[i]]=True

AllAlphasBetas=tmpdict.keys()
for k in AllAlphasBetas:
    listofpoints_alpha.append(k[0])
    listofpoints_beta.append(k[1])
    
    
UnitVectors=[]
d2r=1

# Make 3D unit vectors on the sphere for each point available
for i in range(0, len(listofpoints_alpha)):
    UnitVectors.append([np.sin(listofpoints_alpha[i]*d2r)*np.cos(listofpoints_beta[i]*d2r),np.sin(listofpoints_alpha[i]*d2r)*np.sin(listofpoints_beta[i]*d2r),np.cos(listofpoints_alpha[i]*d2r)]) 

# print(UnitVectors)


In [59]:
# This function finds the closest available (a,b) on the sphere to the one we have provided
def ClosestPoint(a,b):
    ThisUnitVector=[np.sin(a)*np.cos(b),np.sin(a)*np.sin(b),np.cos(a)]
    dotprods=[np.abs(ThisUnitVector[0]*UnitVectors[i][0]+ThisUnitVector[1]*UnitVectors[i][1]+ThisUnitVector[2]*UnitVectors[i][2]) for i in range(0,len(UnitVectors))]
    closest=np.argmax(dotprods)
    return listofpoints_alpha[closest],listofpoints_beta[closest],closest

In [60]:
# Get a continously interpolated F(E) function for nearest a and b
def GetEnergyFunction(a,b,EPower=3):
    ECut=1e5
    besta,bestb,idx=ClosestPoint(a,b)
    dataSubSet=mussimData[(mussimData.alpha==besta) & (mussimData.beta==bestb) & (mussimData.Energies<ECut)]
    interpfun= interpolate.interp1d(dataSubSet.Energies,dataSubSet.MuFLux)
    returnfunction=lambda E: interpfun(E)/E**EPower
    return returnfunction

In [61]:
# Integrate the continuous flux function near to (a,b) between these limits
def GetFluxIntegral(a,b,EMin,EMax):
    EnergyFunction=GetEnergyFunction(a,b)
    return integrate.quad(EnergyFunction,EMin,EMax)

In [62]:
# Energies to get fluxes over. Change this to match the energies Proposal had been run over
# BinCenters=np.array([100,300,400,450,500,550,600, 700,800,1000,2000,4000,6000])
BinCenters=np.array([100, 300, 400, 450, 475, 500, 525, 550, 575, 600, 
                     625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 875, 
                     900, 950,  1000, 1100, 1200, 1300, 1400, 1500, 1600,
                     1700, 1800, 1900, 2000, 3000, 4000, 5000, 6000])


# BinCenters=np.array([100,300,400,450,500,550,600,650,700,800,1000,2000,4000,6000])

EBins = (BinCenters[1:]+BinCenters[:-1])/2
print(len(EBins), len(BinCenters))
print(EBins)

37 38
[ 200.   350.   425.   462.5  487.5  512.5  537.5  562.5  587.5  612.5
  637.5  662.5  687.5  712.5  737.5  762.5  787.5  812.5  837.5  862.5
  887.5  925.   975.  1050.  1150.  1250.  1350.  1450.  1550.  1650.
 1750.  1850.  1950.  2500.  3500.  4500.  5500. ]


In [63]:
# Get muon fluxes vs energy mesh
numofstps = 50 # was 50
cosa_sample = np.linspace(0,1,numofstps)
a_sample = np.arccos(cosa_sample)

b_sample = np.linspace(-np.pi,np.pi,numofstps)
cosa_mesh, b_mesh = np.meshgrid(cosa_sample,b_sample)
fluxmeshes=[]


for iE in range(0,len(EBins)-1):
# for iE in range(0,2):

    fluxmesh = np.zeros_like(cosa_mesh)
    print("Integrating fluxes for "+ str(EBins[iE])+ " < E < " + str(EBins[iE+1]))
    for ia in range(0,len(a_sample)):
        for ib in range(0,len(b_sample)):
            a=a_sample[ia]
            b=b_sample[ib]
            fluxmesh[ia,ib] = GetFluxIntegral(a,b,EBins[iE],EBins[iE+1])[0]
    fluxmeshes.append(fluxmesh)

Integrating fluxes for 200.0 < E < 350.0
Integrating fluxes for 350.0 < E < 425.0
Integrating fluxes for 425.0 < E < 462.5
Integrating fluxes for 462.5 < E < 487.5
Integrating fluxes for 487.5 < E < 512.5
Integrating fluxes for 512.5 < E < 537.5
Integrating fluxes for 537.5 < E < 562.5
Integrating fluxes for 562.5 < E < 587.5
Integrating fluxes for 587.5 < E < 612.5
Integrating fluxes for 612.5 < E < 637.5
Integrating fluxes for 637.5 < E < 662.5
Integrating fluxes for 662.5 < E < 687.5
Integrating fluxes for 687.5 < E < 712.5
Integrating fluxes for 712.5 < E < 737.5
Integrating fluxes for 737.5 < E < 762.5
Integrating fluxes for 762.5 < E < 787.5
Integrating fluxes for 787.5 < E < 812.5
Integrating fluxes for 812.5 < E < 837.5
Integrating fluxes for 837.5 < E < 862.5
Integrating fluxes for 862.5 < E < 887.5
Integrating fluxes for 887.5 < E < 925.0
Integrating fluxes for 925.0 < E < 975.0
Integrating fluxes for 975.0 < E < 1050.0
Integrating fluxes for 1050.0 < E < 1150.0
Integrating f

In [64]:
#save the output
f=open("Files/MCEqFluxMeshes.pkl","wb")
pickle.dump([BinCenters,cosa_sample,b_sample,fluxmeshes],f)
f.close()

In [65]:
#these are for creating the interpolation function between all alpha and energy

k=mussimData.alpha.unique()
k.sort()

df=mussimData[(mussimData.MuFLux>10**-9)]
Es=df.Energies.unique()
Es.sort()
    
alt=1431   
flxs=[] 
for alpha in k:

    for nrg in Es:
        flxs.append(mussimData[(mussimData.alpha==alpha)&(mussimData.Energies==nrg)].MuFLux.iloc[0])           

        
flxarray=np.array(flxs)
flxarray = flxarray.reshape(len(k), len(Es))
FluxMuons=scipy.interpolate.RectBivariateSpline(k,Es,flxarray,s=0,kx=3, ky=3)



In [66]:
#save interpolation function
with open('Files/MCeQ_Muons_interpolator_ALL.pkl', 'wb') as f:
    pickle.dump(FluxMuons, f)